In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [2]:
df = pd.read_csv('./fake_user_cluster.csv')
print(df.columns)
df.head()

Index(['Unnamed: 0', 'phone_number', 'Activities', 'kharif_crops',
       'Farmertype', 'district', 'Own_animal_large', 'Own_animal_small',
       'Own_poultry', 'cluster'],
      dtype='object')


,Unnamed: 0,phone_number,Activities,kharif_crops,Farmertype,district,Own_animal_large,Own_animal_small,Own_poultry,cluster
0,859901,7894892401,['Crop_prod'],"['paddy', 'brinjal', 'blackgram', 'greengram']",NaN,KEONJHAR,NaN,NaN,NaN,2
1,2760612,9938189160,['Animal_husbandry'],NaN,NaN,BOUDH,NaN,NaN,NaN,18
2,2749012,9938089536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,2439876,9777234388,['Crop_prod'],NaN,NaN,BALASORE,NaN,NaN,NaN,2
4,2384312,9776210885,"['Crop_prod', 'Animal_husbandry']","['cabbage', 'paddy', 'brinjal']",3.0,KEONJHAR,NaN,NaN,NaN,2


In [4]:
df = df.drop(columns=['Unnamed: 0', 'phone_number','Farmertype', 'district', 'Own_animal_large', 'Own_animal_small','Own_poultry'],axis=1)

In [20]:
df = df.fillna("[]")

In [21]:
df.head()

,Activities,kharif_crops,cluster
0,['Crop_prod'],"['paddy', 'brinjal', 'blackgram', 'greengram']",2
1,['Animal_husbandry'],[],18
2,[],[],2
3,['Crop_prod'],[],2
4,"['Crop_prod', 'Animal_husbandry']","['cabbage', 'paddy', 'brinjal']",2


In [30]:
def convert_to_list(x):
    tmp = [y.strip() for y in x.strip('][').replace("'","").split(',')]
    if len(tmp[0])==0:
        return []
    return tmp

In [34]:
def find_uniques(lst):
    uniq = set()
    for itm in lst:
        itm = set(convert_to_list(itm))
        uniq.update(itm)
    return uniq

In [35]:
activities = find_uniques(df['Activities'].to_list())
khareef_crops = find_uniques(df['kharif_crops'].to_list())

In [38]:
all_features = list(activities) + list(khareef_crops)
feat_to_index = {f:i for i,f in enumerate(all_features)}

In [43]:
user_data = np.zeros((len(df),len(all_features)))

In [48]:
for idx,row in df.iterrows():
    act = convert_to_list(row['Activities'])
    khr = convert_to_list(row['kharif_crops'])
    for itm in act+khr:
        itm_idx = feat_to_index[itm]
        user_data[idx,itm_idx] = 1

In [57]:
data = np.hstack([df['cluster'].to_numpy().reshape(-1,1),user_data])
all_columns = ['cluster']+all_features
new_df = pd.DataFrame(data,columns=all_columns)

In [60]:
user_cluster = new_df.groupby(['cluster']).sum()

In [62]:
# for crops find fraction
for col in list(user_cluster.columns)[4:]:
    user_cluster[col] = user_cluster[col]/sum(user_cluster[col])

In [72]:
def f(x):
    curr_values = user_cluster.iloc[x].to_dict()
    fig = plt.figure(figsize=(20,10))
    for i in range(1,5):
        plt.subplot(2,5,i+5)
        col,col_value = list(curr_values.keys())[i-1],list(curr_values.values())[i-1]
        other = user_cluster[col].sum() - col_value
        y = [col_value,other]
        labels = [col,"Other"]
        explode = [0.2,0]
        plt.pie(y, labels = labels, explode = explode)
        plt.title(f'Columns {col}')
        
    plt.subplot(2,5,10)
    labels,y = list(curr_values.keys())[:5],list(curr_values.values())[:5]
    explode = [0.2]*len(labels)
    plt.pie(y, labels = labels, explode = explode)
    plt.title(f'Activities')
        
    plt.subplot(2, 1, 1)
    plt.plot(list(curr_values.keys())[4:],list(curr_values.values())[4:])
    plt.title(f'khareef crops vs fraction for cluster {x}')
    plt.xlabel('Crop')
    plt.ylabel('Fraction')
    plt.xticks(rotation=90)

In [73]:
from ipywidgets import interact
import ipywidgets as widgets
f = interact(f,x=widgets.IntSlider(min=0, max=20, step=1, value=0))

interactive(children=(IntSlider(value=0, description='x', max=20), Output()), _dom_classes=('widget-interact',…